In [1]:
from src.strategies import RandomSubset

TypeError: Cannot create a consistent method resolution
order (MRO) for bases SequenceClassificationMixin, UncertaintyStrategy

In [1]:
import json
from pathlib import Path

import hnswlib as hb
import numpy as np
import pandas as pd
import requests
import srsly
from datasets import (
    ClassLabel,
    Dataset,
    DatasetDict,
    Features,
    Sequence,
    Value,
    load_dataset,
    load_from_disk,
)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from tqdm.auto import tqdm

In [62]:
index_path = (
    "/home/pl487/allset/data/processed/eurlex-57k/all-MiniLM-L12-v2_cosine.bin"
)
meta_path = (
    "/home/pl487/allset/data/processed/eurlex-57k/all-MiniLM-L12-v2_cosine.json"
)

In [63]:
meta = srsly.read_json(meta_path)
index = hb.Index(space=meta["metric"], dim=meta["dim"])
index.load_index(str(index_path))

In [69]:
index.mark_deleted(0)

In [70]:
np.stack(index.get_items([0])).shape

RuntimeError: Label not found

In [73]:
sorted(index.get_ids_list())

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [38]:
ds_dict = load_from_disk("/home/pl487/allset/data/prepared/eurlex-57k_bert-tiny")

In [60]:
ds_dict["train"].to_pandas()["input_ids"].map(len).describe()

count    45000.000000
mean       460.139111
std        338.869264
min         19.000000
25%        271.000000
50%        361.000000
75%        524.000000
max       4252.000000
Name: input_ids, dtype: float64

In [45]:
d = load_from_disk("/home/pl487/allset/data/processed/eurlex-57k")

In [61]:
ds_dict

DatasetDict({
    train: Dataset({
        features: ['uid', 'labels', 'text', 'input_ids', 'attention_mask'],
        num_rows: 45000
    })
    test: Dataset({
        features: ['uid', 'labels', 'text', 'input_ids', 'attention_mask'],
        num_rows: 6000
    })
})

In [46]:
df = d["train"].to_pandas()

In [47]:
df.loc[df["uid"] == 40384]

,celex_id,document_type,title,header,recitals,main_body,eurovoc_concepts,text,uid,embedding_all-mpnet-base-v2,embedding_multi-qa-mpnet-base-dot-v1,embedding_all-MiniLM-L12-v2
40384,31994D0413(01),Decision,Council Decision of 29 March 1994 concerning t...,COUNCIL DECISION of 29 March 1994 concerning t...,,[Article 1\nIf Member of the Council represent...,"[114, 127, 1750]",Council Decision of 29 March 1994 concerning t...,40384,"[0.010754533, -0.040741973, 0.006255827, 0.015...","[-0.026998838, -0.30596825, -0.32041094, -0.12...","[0.0583123, 0.0029096473, -0.04903518, 0.02579..."


In [57]:
df["recitals"].str.len().describe()

count    45000.000000
mean      2004.405600
std       1659.948942
min          0.000000
25%       1071.000000
50%       1493.000000
75%       2296.000000
max      19606.000000
Name: recitals, dtype: float64

In [58]:
df["text"].str.len().describe()

count    45000.000000
mean      2223.471911
std       1674.690574
min        105.000000
25%       1283.000000
50%       1727.000000
75%       2529.000000
max      19779.000000
Name: text, dtype: float64

In [2]:
# ds_dict = load_dataset("pietrolesci/eurlex_indexeda", cache_dir="ds_cache")
# ds_dict.save_to_disk("../data/processed/eurlex")

In [20]:
ds_dict = load_from_disk(
    "/home/pl487/allset/data/processed/amazoncat-13k/"
).select_columns(["uid", "target_ind", "text"])

In [21]:
train_df = ds_dict["train"].to_pandas()
test_df = ds_dict["test"].to_pandas()

In [7]:
with open("/home/pl487/allset/AmazonCat-13K.raw/Yf.txt", encoding="latin-1") as fl:
    cats = fl.read().split("\n")
cats = pd.DataFrame(cats)

In [9]:
a = train_df["target_ind"].explode().value_counts(True).to_frame("f")

In [10]:
a.loc[(a["f"] > 0.01) & (a["f"] < 0.03)].sort_values("f", ascending=False).join(cats)

,f,0
target_ind,,
7892,0.021411,movies & tv
9237,0.020084,pop
7083,0.016357,literature & fiction
7891,0.014879,movies
4038,0.012757,education & reference
10063,0.012549,rock
8108,0.011986,new
12630,0.011986,used & rental textbooks


In [32]:
ids = cats.loc[cats[0].str.contains("movie")].tail(3).index.tolist()
cats.loc[cats[0].str.contains("movie")]

,0
7888,movie & tv fan apparel
7889,movie directors
7890,movie tie-ins
7891,movies
7892,movies & tv
7893,movies & video


In [35]:
mask = train_df["target_ind"].map(lambda ex: any(i in ex for i in ids))

In [36]:
train_df.loc[mask]

,uid,target_ind,text
13,13,"[7891, 7892]",Shrek - The Story So Far (Shrek 1 & 2 Full Scr...
26,26,"[7891, 7892]","Mission Impossible [VHS] (1996)\n\nA flashy, s..."
68,68,"[2351, 7892, 7961, 12505]",The Birth of The Constitution:This is America ...
76,76,"[7891, 7892]",Student of Prague Collection (1913 & 1926 Vers...
85,85,"[7892, 12505]",Daily Dose of Dharma with Danica McKellar\n\nA...
...,...,...,...
1186210,1186210,"[7891, 7892]",Riding High (1950)\n\nRIDING HIGH tells the st...
1186219,1186219,"[7891, 7892]",Roy Rogers - Grand Canyon Trail\n\nNo Descript...
1186225,1186225,"[7891, 7892]",Bardot - Et Dieu... créa la femme/Documentaire...
1186230,1186230,"[7892, 12505]",Galloping Minds - Preschooler Learns Numbers a...


In [37]:
131315 / len(train_df)

0.11069860289536931

In [31]:
train_df.loc[train_df["target_ind"].map(lambda ex: 7893 in ex), "text"].to_dict()

{42077: 'Monsters, Mushroom Clouds, and the Cold War: American Science Fiction and the Roots of Postmodernism, 1946-1964 (Contributions to the Study of Science Fiction and Fantasy)\n\n?[a]n impressive, enjoyable book, and one that makes valuable contributions to the history of American science fiction and to our understanding of the culture of the American 1950s. We should be grateful not only to Booker but also--and yet again!--to the much under-appreciated publishers of the Greenwood Press, whose long-time support of sf criticism is as admirable as it is generally unsung.?-Science Fiction Studies\tStudies the engagement of American science fiction novels and films of the long 1950s with their historical and political contexts, the rise of late capitalism, and the birth of postmodernism.\tM. Keith Booker is Professor of English at the University of Arkansas.',
 232008: "Careers for Film Buffs & Other Hollywood Types (Careers For Series)\n\nCareers for Film Buffs  Other Hollywood Types

In [ ]:
# df.loc[df["target_ind"].map(lambda ex: 13199 in ex)]
df.loc[df["target_ind"].map(lambda ex: 12583 in ex)].iloc[2].to_dict()

In [ ]:
30068 / len(df)

In [ ]:
MODELS = ["all-mpnet-base-v2", "multi-qa-mpnet-base-dot-v1", "all-MiniLM-L12-v2"]

In [ ]:
models = [f"embedding_{i}" for i in MODELS]
df = load_dataset("pietrolesci/eurlex_indexed")["train"].to_pandas()
df = df[["uid"] + models]

In [ ]:
metric: str = "cosine"
ef_construction: int = 200
ef: int = 200
M: int = 64
num_threads: int = 5

path = Path("../data/processed/eurlex/hnswlib_indices")
path.mkdir(exist_ok=True, parents=True)

for idx, model in enumerate(tqdm(models)):

    path.mkdir(exist_ok=True, parents=True)

    embeddings = np.stack(df[model].values)

    index = hb.Index(space=metric, dim=embeddings.shape[1])
    index.set_ef(ef)
    index.init_index(
        max_elements=embeddings.shape[0],
        M=M,
        ef_construction=ef_construction,
        random_seed=42,
    )

    index.add_items(embeddings, df["uid"].tolist(), num_threads=num_threads)
    index.save_index(str(path / f"index_{metric}_{MODELS[idx]}.bin"))

    srsly.write_yaml(
        path / f"metadata_{metric}_{MODELS[idx]}.yaml",
        {
            "metric": metric,
            "ef_construction": ef_construction,
            "ef": ef,
            "M": M,
            "num_threads": num_threads,
        },
    )

In [ ]:
data = ds_dict.map(
    lambda ex: {"labels": [int("3191" in l) for l in ex["eurovoc_concepts"]]},
    batched=True,
)

In [ ]:
train_df = data["train"].to_pandas()
test_df = data["test"].to_pandas()

In [ ]:
train_freq = (
    train_df[["celex_id", "eurovoc_concepts"]]
    .explode("eurovoc_concepts")
    .groupby("eurovoc_concepts")["celex_id"]
    .nunique()
    .to_frame("count_train")
)
test_freq = (
    test_df[["celex_id", "eurovoc_concepts"]]
    .explode("eurovoc_concepts")
    .groupby("eurovoc_concepts")["celex_id"]
    .nunique()
    .to_frame("count_test")
)

In [ ]:
desc = pd.DataFrame(
    srsly.read_jsonl("../data/raw/eurlex-57k/eurovoc_concepts.jsonl")
).set_index("id")

In [ ]:
f = train_freq.join(test_freq).dropna()

In [ ]:
f["sum"] = f.sum(1)

In [ ]:
f = f.sort_values("sum", ascending=False)

In [ ]:
f = f.join(desc)

In [ ]:
f.loc[(f["sum"] < 3500) & (f["sum"] > 1000)].sort_values(
    ["count_test", "count_train"], ascending=[False, True]
)

In [ ]:
# make `health control` the target label
data = ds_dict.map(
    lambda ex: {"labels": [int("192" in l) for l in ex["eurovoc_concepts"]]},
    batched=True,
)
train_df = data["train"].to_pandas()
test_df = data["test"].to_pandas()

In [ ]:
train_df["labels"].value_counts(True), test_df["labels"].value_counts(True)

In [ ]:
tfidf = TfidfVectorizer(ngram_range=(1, 3))
log_reg = LogisticRegression()

In [ ]:
train_df["document_type"].value_counts()

In [ ]:
# X = tfidf.fit_transform(train_df["text"].values)
X = np.stack(train_df["embedding_all-mpnet-base-v2"].values)
y = train_df["labels"].values
# y = train_df["document_type"] == "Directive"

# X_test = tfidf.transform(test_df["text"].values)
X_test = np.stack(test_df["embedding_all-mpnet-base-v2"].values)
y_test = test_df["labels"].values
# y_test = test_df["document_type"] == "Directive"

In [ ]:
log_reg.fit(X, y)

In [ ]:
print(classification_report(y_test, log_reg.predict(X_test)))

In [ ]:
print(classification_report(y, log_reg.predict(X)))

In [ ]:
ds_dict

In [ ]:
1609 / len(train_df)

In [ ]:
train_df["document_type"].value_counts(True)

In [ ]:
train_df.loc[train_df["eurovoc_concepts"].map(lambda ex: "2173" in ex)]

In [ ]:
desc

In [ ]:
# get eurlex from http://nlp.cs.aueb.gr/software_and_datasets/EURLEX57K/ and unzip the folder into eurlex
path = Path("../eurlex/")
data = {}
for split in path.iterdir():
    data[split.name] = []
    for file in tqdm(list(split.iterdir())):
        data[split.name].append(srsly.read_json(file))

In [ ]:
cats = [
    "Wireless_v1_00",
    "Watches_v1_00",
    "Video_Games_v1_00",
    "Video_DVD_v1_00",
    "Video_v1_00",
    "Toys_v1_00",
    "Tools_v1_00",
    "Sports_v1_00",
    "Software_v1_00",
    "Shoes_v1_00",
    "Pet_Products_v1_00",
    "Personal_Care_Appliances_v1_00",
    "PC_v1_00",
    "Outdoors_v1_00",
    "Office_Products_v1_00",
    "Musical_Instruments_v1_00",
    "Music_v1_00",
    "Mobile_Electronics_v1_00",
    "Mobile_Apps_v1_00",
    "Major_Appliances_v1_00",
    "Luggage_v1_00",
    "Lawn_and_Garden_v1_00",
    "Kitchen_v1_00",
    "Jewelry_v1_00",
    "Home_Improvement_v1_00",
    "Home_Entertainment_v1_00",
    "Home_v1_00",
    "Health_Personal_Care_v1_00",
    "Grocery_v1_00",
    "Gift_Card_v1_00",
    "Furniture_v1_00",
    "Electronics_v1_00",
    "Digital_Video_Games_v1_00",
    "Digital_Video_Download_v1_00",
    "Digital_Software_v1_00",
    "Digital_Music_Purchase_v1_00",
    "Digital_Ebook_Purchase_v1_00",
    "Camera_v1_00",
    "Beauty_v1_00",
    "Baby_v1_00",
    "Automotive_v1_00",
    "Apparel_v1_00",
    "Digital_Ebook_Purchase_v1_01",
    "Books_v1_00",
    "Books_v1_01",
    "Books_v1_02",
]

In [ ]:
ds_dict = load_dataset("pietrolesci/pubmed-rct-200k_indexed")
ds_dict = ds_dict.select_columns(
    ["uid", "labels", "text", "embedding_all-mpnet-base-v2"]
)

train_df = ds_dict["train"].to_pandas()
test_df = ds_dict["test"].to_pandas()

In [ ]:
X = np.stack(train_df["embedding_all-mpnet-base-v2"].values)
y = train_df["labels"].values == 0

X_test = np.stack(test_df["embedding_all-mpnet-base-v2"].values)
y_test = test_df["labels"].values == 0

In [ ]:
train_df["labels"].value_counts(True)

In [ ]:
log_reg = LogisticRegression()

In [ ]:
log_reg.fit(X, y)

In [ ]:
print(classification_report(y_test, log_reg.predict(X_test)))

In [ ]:
print(classification_report(y, log_reg.predict(X)))

In [ ]:
len(train_df)

In [ ]:
d = load_dataset("pietrolesci/pubmed-rct-200k_indexed")

In [ ]:
a